In [11]:
#uncomment and run the below line to install geopy
#%conda install --name metis -c conda-forge geopy

In [12]:
import sys
import pandas as pd
import pickle
from geopy import distance

In [13]:
#read from NYC OpenData
CU_data = pd.read_csv('https://data.cityofnewyork.us/api/views/8pnn-kkif/rows.csv', 
                      usecols=['the_geom', 'NAME', 'ZIP'])
#remove POINT and parentheses
CU_data['the_geom'] = CU_data['the_geom'].map(lambda x: str(x)[7:])
CU_data['the_geom'] = CU_data['the_geom'].map(lambda x: str(x)[:-1])
#separate lat and long columns
CU_data['Long'] = CU_data.the_geom.str.split('\s+').str[0]
CU_data['Lat'] = CU_data.the_geom.str.split('\s+').str[-1]
#drop 'the_geom' column
CU_data = CU_data.drop('the_geom', axis=1)

In [14]:
CU_data.head()

,NAME,ZIP,Long,Lat
0,New School University / Parsons School Of Design,10011,-73.99465215457163,40.73519616365903
1,New School University / The New School,10011,-73.99706966379965,40.73546280987431
2,Metropolitan College,10013,-74.00681944352681,40.723441859057495
3,School Of Nursing Of St. Vincents Catholic Med...,11365,-73.79439300079635,40.73944287003665
4,Bramson Ort College,11375,-73.84872054010768,40.721371886956774


In [16]:
with open('pickle/station_lookup.pickle','rb') as read_file: #run stop_id_matchup to generate
    station_lookup = pickle.load(read_file)

In [17]:
station_lookup.head()

,C/A,STATION,stop_id,Stop Name,GTFS Latitude,GTFS Longitude,Borough,Daytime Routes
0,A002,59 ST,R11,Lexington Av/59 St,40.762660,-73.967258,M,N W R
1,A006,5 AV/59 ST,R13,5 Av/59 St,40.764811,-73.973347,M,N W R
2,A007,5 AV/59 ST,R13,5 Av/59 St,40.764811,-73.973347,M,N W R
3,A010,57 ST-7 AV,R14,57 St - 7 Av,40.764664,-73.980658,M,N Q R W
4,A011,57 ST-7 AV,R14,57 St - 7 Av,40.764664,-73.980658,M,N Q R W


In [18]:
station_CU_proximity = station_lookup
station_CU_proximity = station_CU_proximity.drop_duplicates('STATION')

In [19]:
def isclose(coords1, coords2, maxdistance):
    '''Calcluate the distance between two Latitude/Longitude coordinates, must have GeoPy installed and imported
    
    Args:
        coords1 = list object of coordinates (Latitude -- Longitude) of first point
        coords2 = list object of coordinates (Latitude -- Longitude) of second point
        maxdistance = float or int value of the maximum distance between coords1 and coords2 in miles
    
    Returns:
        0 = not within maxdistance
        1 = within max distance
    '''
    dist = distance.distance(
        (coords1[0], coords1[1]),
        (coords2[0], coords2[1])).miles
    if dist < maxdistance:
        return 1
    else:
        return 0

In [20]:
station_CU_proximity['coord'] = list(zip(station_CU_proximity['GTFS Latitude'],
                                         station_CU_proximity['GTFS Longitude']))
CU_data['coord'] = list(zip(CU_data['Lat'],
                            CU_data['Long']))

/Users/alexanderparker/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
station_CU_proximity['SCHOOL COUNT'] = station_CU_proximity['coord'].apply(lambda x: CU_data['coord'].apply(
    isclose, args=[x,0.75]).sum())

/Users/alexanderparker/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
station_CU_proximity.sort_values(by='SCHOOL COUNT', ascending=False).reset_index()

,index,C/A,STATION,stop_id,Stop Name,GTFS Latitude,GTFS Longitude,Borough,Daytime Routes,coord,SCHOOL COUNT
0,14,A030,23 ST,R19,23 St,40.741303,-73.989344,M,R W,"(40.741303, -73.98934399999999)",11
1,16,A033,14 ST-UNION SQ,L03,Union Sq - 14 St,40.734789,-73.990730,M,L,"(40.734789, -73.99073)",11
2,121,H005,3 AV,L05,3 Av,40.732849,-73.986122,M,L,"(40.732849, -73.986122)",10
3,120,H003,6 AV,L02,6 Av,40.737335,-73.996786,M,L,"(40.737334999999995, -73.996786)",10
4,12,A027,28 ST,R18,28 St,40.745494,-73.988691,M,R W,"(40.745494, -73.988691)",9
5,473,R121,HOUSTON ST,134,Houston St,40.728251,-74.005367,M,1,"(40.728251, -74.005367)",9
6,442,PTH09,CHRISTOPHER ST,133,Christopher St - Sheridan Sq,40.733422,-74.002906,M,1,"(40.733422, -74.00290600000001)",8
7,197,N029,116 ST,A16,116 St,40.805085,-73.954882,M,B C,"(40.805085, -73.954882)",8
8,227,N078,14 ST,A31,14 St,40.740893,-74.001690,M,A C E,"(40.740893, -74.00169)",8
9,228,N080,W 4 ST-WASH SQ,A32,W 4 St,40.732338,-74.000495,M,A C E,"(40.732338, -74.000495)",8


In [23]:
with open('pickle/station_CU_proximity.pickle', 'wb') as to_write:
    pickle.dump(station_CU_proximity, to_write)